In [75]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from datetime import date
from pathlib import Path

In [76]:
options = Options()
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.77 Safari/537.36")

driver = webdriver.Chrome(options=options)

In [77]:
# Load ridership page from BMRCL website
driver.get("https://english.bmrc.co.in/ridership/")
driver.find_element(By.CLASS_NAME, "link.top-navcustom-text").click()

In [78]:
record_date = driver.find_element(By.TAG_NAME, "h3").text
record_date

'Passenger Flow as on 27-10-2024'

In [79]:
day_record = {}
day_record['Record Date'] = list()

day_record['Record Date'].append(record_date.split()[-1])
day_record

{'Record Date': ['27-10-2024']}

In [80]:
data_points = driver.find_elements(By.CLASS_NAME, "features-card.achivement-area.bg-color")

for l1 in data_points:
    for l2 in l1.text.split('\n'):
        data = l2.split(': ')
        day_record[data[0]] = list()
        day_record[data[0]].append(int(data[1]))

day_record = pd.DataFrame(day_record)
day_record

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
0,27-10-2024,176278,175195,971,75,37,250124,4278,458,205910,56321,111959,37630


In [83]:
filename = "NammaMetro_Ridership_Dataset.csv"
filePath = Path(filename)

if filePath.exists() and filePath.is_file():
    day_record.to_csv(filename, mode='a', header=False)
else:
    day_record.to_csv(filename, mode='a', columns=day_record.keys(), header=True)


In [87]:
# Optimize dataset by removing duplicates
df = pd.read_csv(filename).drop_duplicates(keep='last', ignore_index=True).loc[:,'Record Date':]
df.to_csv(filename, mode='w', columns=day_record.keys(), header=True)
df

,Record Date,Total Smart Cards,Stored Value Card,One Day Pass,Three Day Pass,Five Day Pass,Tokens,Total NCMC,Group Ticket,Total QR,QR NammaMetro,QR WhatsApp,QR Paytm
0,26-10-2024,353460,352496,853,43,68,241883,7444,512,177279,49351,95571,32357.0
1,27-10-2024,176278,175195,971,75,37,250124,4278,458,205910,56321,111959,37630.0
